In [ ]:
%pip install ipython
%pip install numpy
%pip install pandas
%pip install nbformat
%pip install statsmodels
%pip install mlforecast
%pip install lightgbm
%pip install sktime
%pip install scikit-learn
%pip install matplotlib
%pip install plotly
%pip install holidays

%pip install tensorflow
%pip install tensorboard

In [1]:
import re
import numpy as np
import pandas as pd

from tqdm import tqdm
import holidays
th_holidays = holidays.TH()

import plotly.express as px
import plotly.io as pio
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
physical_device = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_device[0], True)

from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout, Flatten, Input, MultiHeadAttention, LayerNormalization, Concatenate, concatenate
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.models import load_model


# Helper functions
def if_work_day(date): 
    if ((date in th_holidays) or (date.weekday()>=5)):
        return 0
    else: return 1
    
def reshape_ts_X(input_vector, time_steps, batch_size=None):
    x = np.lib.stride_tricks.sliding_window_view(input_vector[:-1], 
            window_shape=(time_steps, * input_vector.shape[1:]))
    x = x.reshape(-1, time_steps, * input_vector.shape[1:]) # removes extra dimension
    if (batch_size is not None) and (batch_size > 1):
        x = np.array_split(x, len(x) // batch_size)
    # else: x = x[np.newaxis, :, :] 
    return x

def reshape_ts_Y(target_vector, time_steps, batch_size=None):
    y = target_vector[time_steps:]
    if (batch_size is not None) and (batch_size > 1):
        y = np.array_split(y, len(y) // batch_size)
    # else: y = y[np.newaxis, :]
    return y

In [2]:
grayscale_template = {
    "layout": {
        "paper_bgcolor": "white",  # Background color
        "plot_bgcolor": "white",   # Plot background color
        "font": {
            "color": "black"  # Text color
        },
        "xaxis": {
            "linecolor": "black",  # X-axis line color
            "gridcolor": "lightgray",  # X-axis grid color
            "zerolinecolor": "gray"    # X-axis zero line color
        },
        "yaxis": {
            "linecolor": "black",  # Y-axis line color
            "gridcolor": "lightgray",  # Y-axis grid color
            "zerolinecolor": "gray"    # Y-axis zero line color
        }
    }
}

# Set the default template to the grayscale template
pio.templates["grayscale"] = grayscale_template
pio.templates.default = "grayscale"

#### Data import

In [3]:
merged_df = pd.read_csv('data/clean_merged_dataset.csv',parse_dates = ['Date']).set_index('Date')

# Combine by type (Light, Plug, AC)
merged_df.columns = [re.sub(r"\d+$", "", column) for column in merged_df.columns]
merged_df = merged_df.groupby(by=merged_df.columns, axis=1).sum()

tuple_columns = [(x[0][-1], x[1][-1], x[2]) for x in [column.split('_') for column in merged_df.columns]]
merged_df.columns = pd.MultiIndex.from_tuples(tuple_columns, names=['Floor', 'Zone','Sensor'])


merged_df = merged_df.resample('D').sum() 
merged_df = merged_df.iloc[:-2] # For clean weeks in testing

total_load_df = merged_df.sum(axis=1)
total_by_zone_df = merged_df.groupby(level=1, axis = 1).sum()
total_by_type_df = merged_df.groupby(level=2, axis = 1).sum()

test_len = 7*37

#### DATA BY TOTAL

In [4]:
### DATA BY TOTAL

data = total_load_df.reset_index()
data.columns = ['Date', 'Total']
data['If Work Day'] = data['Date'].map(if_work_day)
X = data.iloc[:, 1:].to_numpy()
next_if_work_day = data['If Work Day'].to_numpy().reshape(-1, 1)
y = np.array(data[['Total']])

scaler = MinMaxScaler()
scaler.fit(X)
X_std = scaler.transform(X)
X_std

time_steps = 7

X_train, X_test = X_std[:-test_len], X_std[-test_len-time_steps:]

next_if_work_day_train = next_if_work_day[:-test_len]
next_if_work_day_test = next_if_work_day[-test_len-time_steps:]
    
y_train, y_test = y[:-test_len], y[-test_len-time_steps:]

X_train_batched_ts = reshape_ts_X(X_train, time_steps)
next_if_work_day_train_ts = reshape_ts_Y(next_if_work_day_train, time_steps)
y_train_batched_ts = reshape_ts_Y(y_train, time_steps)

X_test_batched_ts = reshape_ts_X(X_test, time_steps, 7)
next_if_work_day_test_ts = reshape_ts_Y(next_if_work_day_test, time_steps, 7)
y_test_batched_ts = reshape_ts_Y(y_test, time_steps, 7)

#### DATA BY ZONE

In [4]:
### DATA BY ZONE

data = total_by_zone_df.reset_index()
data['If Work Day'] = data['Date'].map(if_work_day)
X = data.iloc[:, 1:].to_numpy()
next_if_work_day = data['If Work Day'].to_numpy().reshape(-1, 1)
y = data.iloc[:, 1:-1].to_numpy()


scaler = MinMaxScaler()
scaler.fit(X)
X_std = scaler.transform(X)
X_std

time_steps = 7

X_train, X_test = X_std[:-test_len], X_std[-test_len-time_steps:]

next_if_work_day_train = next_if_work_day[:-test_len]
next_if_work_day_test = next_if_work_day[-test_len-time_steps:]
    
y_train, y_test = y[:-test_len], y[-test_len-time_steps:]


X_train_batched_ts = reshape_ts_X(X_train, time_steps)
next_if_work_day_train_ts = reshape_ts_Y(next_if_work_day_train, time_steps)
y_train_batched_ts = reshape_ts_Y(y_train, time_steps)

X_test_batched_ts = reshape_ts_X(X_test, time_steps, 7)
next_if_work_day_test_ts = reshape_ts_Y(next_if_work_day_test, time_steps, 7)
y_test_batched_ts = reshape_ts_Y(y_test, time_steps, 7)

#### DATA BY TYPE

In [ ]:
### DATA BY TYPE

data = total_by_type_df.reset_index()
data['If Work Day'] = data['Date'].map(if_work_day)
X = data.iloc[:, 1:].to_numpy()
next_if_work_day = data['If Work Day'].to_numpy().reshape(-1, 1)
y = np.array(data[['AC', 'Light', 'Plug']])

scaler = MinMaxScaler()
scaler.fit(X)
X_std = scaler.transform(X)
X_std

time_steps = 7

X_train, X_test = X_std[:-test_len], X_std[-test_len-time_steps:]

next_if_work_day_train = next_if_work_day[:-test_len]
next_if_work_day_test = next_if_work_day[-test_len-time_steps:]
    
y_train, y_test = y[:-test_len], y[-test_len-time_steps:]

X_train_batched_ts = reshape_ts_X(X_train, time_steps)
next_if_work_day_train_ts = reshape_ts_Y(next_if_work_day_train, time_steps)
y_train_batched_ts = reshape_ts_Y(y_train, time_steps)

X_test_batched_ts = reshape_ts_X(X_test, time_steps, 7)
next_if_work_day_test_ts = reshape_ts_Y(next_if_work_day_test, time_steps, 7)
y_test_batched_ts = reshape_ts_Y(y_test, time_steps, 7)

#### MODEL - LSTM

In [20]:
# MODEL - LSTM

input1 = Input(shape=(time_steps, X.shape[1]))
input2 = Input(shape=(1,))  

lstm1 = LSTM(units=128, return_sequences=True)(input1)
lstm1 = Dropout(0.2)(lstm1)

lstm2 = LSTM(units=64, return_sequences=True)(lstm1)
lstm2 = Dropout(0.2)(lstm2)

lstm3 = LSTM(units=32, return_sequences=True)(lstm2)
lstm3 = Dropout(0.2)(lstm3)

lstm4 = LSTM(units=64, return_sequences=True)(lstm3)
lstm4 = Dropout(0.2)(lstm4)

lstm5 = LSTM(units=32, return_sequences=False)(lstm4)
lstm5 = Dropout(0.2)(lstm5)

fl_1 = Flatten()(lstm5)
concatenated = concatenate([fl_1, input2])

dense_1 = Dense(units=16)(concatenated)
dense_2 = Dense(units=8)(dense_1)
dense_3 = Dense(units=5)(dense_2)
output = Dense(units=y.shape[1])(dense_3)

model = Model(inputs=[input1, input2], outputs=output)

model.compile(loss='mae', optimizer=Adam(), metrics=['mape'])

#### MODEL - Transformer

In [5]:
### MODEL - Transformer

# Define the input layer
input_ts = Input(shape=(time_steps, X.shape[1]))
input_static = Input(shape=(1,))

# Transformer block
x = MultiHeadAttention(num_heads=8, key_dim=128)(input_ts, input_ts)
x = Dropout(0.2)(x)
x = LayerNormalization(epsilon=1e-6)(x)

fl_1 = Flatten()(x)
fl_2 = Flatten()(input_static)
x = Concatenate(axis=-1)([fl_1, fl_2])

# Additional dense layers
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)

# Output layer
output = Dense(units=y.shape[1])(x)

# Create and compile the model
model = Model(inputs=[input_ts, input_static], outputs=output)
model.compile(loss='mae', optimizer='adam', metrics=['mape'])

In [8]:
# ### WEEKLY PREDICT

# combined_history = {'loss': [], 'mape': []}
# history = model.fit(
#         x=[X_train_batched_ts, next_if_work_day_train_ts],
#         y=y_train_batched_ts,
#         epochs=4000,
#         verbose=1) # TODO: change epochs
# for key in combined_history.keys():
#     combined_history[key].extend(history.history[key])

# X_lagged_input = X_train_batched_ts[-1].tolist()
# if_work_day_input = next_if_work_day_train_ts[-1]

predictions = []
for t_week in tqdm(range(int(test_len/7)), desc="Processing Weeks"):
    for t_day in range(7):
        # Predict next day
        next_pred = model([np.expand_dims(X_lagged_input, axis=0), 
                           np.expand_dims(if_work_day_input, axis=0)]).numpy().tolist()
        predictions.append(next_pred)
        
        # Update input
        input_X = next_pred[0] + next_if_work_day_test[t_week*7+t_day].tolist()
        X_lagged_input = X_lagged_input[1:] + scaler.transform([input_X]).tolist()
        
        if_work_day_input = next_if_work_day_test[t_week*7+t_day+1]
    
    # Train on current week
    history = model.fit(
        x=[X_test_batched_ts[t_week], next_if_work_day_test_ts[t_week]],
        y=y_test_batched_ts[t_week],
        epochs=400,
        verbose=0) # TODO: change epochs
    for key in combined_history.keys():
        combined_history[key].extend(history.history[key])
        print(f"{key} - {combined_history[key][-1]}")
        
history_df = pd.DataFrame(combined_history)
px.line(history_df['loss'], title='Loss').show()
px.line(history_df['mape'], title='Mape').show()
px.line(np.array(predictions).sum(axis=2), title='Prediction').show()

Processing Weeks:   3%|▎         | 1/37 [00:08<05:02,  8.40s/it]

loss - 54.82292556762695
mape - 24.45538330078125


Processing Weeks:   5%|▌         | 2/37 [00:15<04:26,  7.61s/it]

loss - 67.61811828613281
mape - 12.377725601196289


Processing Weeks:   8%|▊         | 3/37 [00:21<03:54,  6.91s/it]

...

loss - 44.69401931762695
mape - 6.034533977508545


Processing Weeks:  97%|█████████▋| 36/37 [04:57<00:08,  8.03s/it]

loss - 62.73748779296875
mape - 7.889557361602783


Processing Weeks: 100%|██████████| 37/37 [05:06<00:00,  8.28s/it]

loss - 77.92162322998047
mape - 8.14619255065918
